In [98]:
import torch
import numpy as np
import os
import json
import sys
sys.path.insert(1,"../")
print(os.getcwd())
from src.models.Palazcnn import PalazCNN
from src.models.lit import Lit
from src.data.nccrmeerkatsdataset import NCCRMeerkatsDataset
from src.data.isabelmeerkatdataset import isabelMerkatDataset
from torch.utils.data import DataLoader, ConcatDataset
import pandas as pd
from src.utils import utils
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn import svm


/remote/idiap.svm/project.evolang/meerkats_imen/evolang_meerkats_calls_classification/workspace/Notebooks-others


In [119]:
MAIN_DIR="/idiap/project/evolang/meerkats_imen/evolang_meerkats_calls_classification/experiments/" #dir for checkpoints
DATA_DIR="/idiap/project/evolang/meerkats_imen/dataset/isabel_data/info_file.csv" # to change depend on data

Extract last layer of best cnn model

In [123]:

lit=Lit(model=PalazCNN(n_input=1,n_output=6),num_classes=6,learning_rate=0.003)

checkpoint=torch.load(MAIN_DIR + "experiment_3/checkpoints/epoch=99-step=9600-v1.ckpt",map_location=torch.device('cpu')) #best marta model
lit.load_state_dict(checkpoint["state_dict"])
lit.model.eval()
with open('../src/data/class_to_index_isabel.json') as f:
        class_to_index = json.load(f)
data_test = isabelMerkatDataset(
            audio_dir=DATA_DIR,
            class_to_index=class_to_index,
            target_sample_rate=16000,
            train=False) 
   
    
data_train = isabelMerkatDataset(
        audio_dir=DATA_DIR,
        class_to_index=class_to_index,
        target_sample_rate=16000,
        train=True) 
result={}
k_folds=5
dataset=torch.utils.data.ConcatDataset([data_test,data_train])
labels=dataset.datasets[0].filelist.class_index.tolist()+ dataset.datasets[1].filelist.class_index.tolist()
kfold=StratifiedKFold(n_splits=k_folds,shuffle=True,random_state=42)

accuracies_folds=[]

for fold,(train_ids,test_ids) in enumerate(kfold.split(dataset,labels)):
        if fold==1:
                
                num_train = len(train_ids)
                split = int(np.floor(0.2* num_train))
                                        #numpy.random.shuffle(indices)

                                        # mask=dataset.filelist.index.isin(train_ids)
                                        # dataset_=dataset.filelist[mask] 
                                        # all_ids=numpy.arange(0,len(dataset))
                                        # mask=x.index.isin(train_ids)
                                        # dataset_=x[mask]
                                        # for k, (train_idx, valid_idx) in enumerate(kfold.split(dataset_,dataset_.class_index)):
                                        #         train_ids=train_idx
                                        #         break
                                        # all_ids=numpy.arange(0,len(dataset))
                                        # keep_mask=numpy.isin(all_ids,test_ids,invert=True)
                                        # new_ids=all_ids[keep_mask]
                                        # train_ids=new_ids[train_ids]
                                        # valid_idx=new_ids[valid_idx]
                print(f'There are {len(test_ids)} data points in the test set ')
                print(f'There are {len(train_ids)} data points in the train set') 
                train_subsampler=torch.utils.data.SubsetRandomSampler(train_ids)
                test_subsampler=torch.utils.data.SubsetRandomSampler(test_ids)

                                # Dataloader in this fold
                train_loader = DataLoader(dataset, batch_size=1,sampler=train_subsampler,collate_fn=utils.collate_fn, num_workers=4)
                test_loader = DataLoader(dataset, batch_size=1,sampler=test_subsampler, shuffle=False,collate_fn=utils.collate_fn, num_workers=4)

                FEATS=[] # list of all the features
                for batch in iter(test_loader):
                        input,labels=batch
                # forward pass [with feature extraction]
                        x,intermediate=lit.model(input)
                        output=np.c_[intermediate.detach().numpy(),labels.detach().numpy()]
                        FEATS.append(output)
                        
                FEATS_test=np.array(FEATS)
                FEATS_test=FEATS_test.reshape((FEATS_test.shape[0]*FEATS_test.shape[1]), FEATS_test.shape[2]) # final features numpy
                FEATS=[]
                for batch in iter(train_loader):
                        input,labels=batch
                # forward pass [with feature extraction]
                        x,intermediate=lit.model(input)
                        output=np.c_[intermediate.detach().numpy(),labels.detach().numpy()]
                        FEATS.append(output)
                        
                FEATS_train=np.array(FEATS)
                print(FEATS_train[:,-1])
                FEATS_train=FEATS_train.reshape((FEATS_train.shape[0]*FEATS_train.shape[1]), FEATS_train.shape[2]) # final features numpy



There are 476 data points in the test set 
There are 1905 data points in the train set


/idiap/home/ibmahmoud/.local/lib/python3.9/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[[ 0.          0.34437376  4.29572678 ...  0.4910565  13.81079292
   2.        ]
 [ 0.          0.04206668  0.27667645 ...  0.27937749  0.71499008
   5.        ]
 [ 0.          0.45979637  0.30111942 ...  0.15423302  3.19676852
   2.        ]
 ...
 [ 0.          0.35824788  0.09588763 ...  0.15609115  0.9584164
   1.        ]
 [ 0.          2.4225142   0.83061194 ...  1.16252112  5.30768347
   2.        ]
 [ 0.          0.40227491  0.05811814 ...  0.37099299  1.23499727
   2.        ]]


In [124]:
param_grid = {
    "n_estimators": [0,10,20,30,40,50, 70,80, 100, 150, 200],
    "max_depth": [None, 5, 7 , 10],
    "min_samples_split": [2, 3,5, 7,10],
    "min_samples_leaf": [1, 2, 3,4]
}


In [125]:
features_test=FEATS_test[:,:-1]
labels_test=FEATS_test[:,-1]
features_train=FEATS_train[:,:-1]
labels_train=FEATS_train[:,-1]


In [126]:
print(FEATS_test.shape)
print(labels_train)

(476, 81)
[2. 5. 2. ... 1. 2. 2.]


RANDOM FOREST

In [127]:
rf=RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(features_train, labels_train)


/idiap/temp/ibmahmoud/miniconda3/envs/s3prl-pytorch/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
400 fits failed out of a total of 4400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
400 fits failed with the following error:
Traceback (most recent call last):
  File "/idiap/temp/ibmahmoud/miniconda3/envs/s3prl-pytorch/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/idiap/temp/ibmahmoud/miniconda3/envs/s3prl-pytorch/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/idiap/temp/ibmahmoud/miniconda3/envs/s3prl-pytorch/lib/pytho

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 7, 10],
                         'min_samples_leaf': [1, 2, 3, 4],
                         'min_samples_split': [2, 3, 5, 7, 10],
                         'n_estimators': [0, 10, 20, 30, 40, 50, 70, 80, 100,
                                          150, 200]})

In [128]:
best_model=grid_search.best_estimator_
y_pred=best_model.predict(features_test)

In [129]:
y_pred


array([2., 5., 5., 2., 1., 2., 5., 5., 5., 2., 5., 2., 2., 2., 5., 1., 2.,
       5., 2., 2., 2., 2., 2., 2., 2., 5., 5., 5., 2., 2., 2., 5., 2., 2.,
       2., 2., 5., 5., 2., 2., 2., 5., 2., 5., 2., 5., 1., 2., 2., 5., 5.,
       5., 2., 5., 2., 2., 5., 2., 5., 5., 5., 2., 2., 5., 1., 2., 2., 2.,
       2., 5., 2., 1., 1., 2., 5., 5., 2., 5., 1., 5., 2., 2., 2., 2., 5.,
       5., 2., 2., 5., 5., 2., 2., 5., 2., 2., 5., 2., 5., 2., 5., 5., 5.,
       2., 5., 1., 2., 2., 5., 2., 2., 2., 2., 5., 2., 5., 5., 2., 5., 2.,
       5., 2., 5., 1., 2., 2., 5., 2., 2., 5., 2., 5., 2., 2., 5., 2., 2.,
       2., 2., 5., 2., 2., 2., 2., 2., 2., 5., 2., 5., 5., 5., 2., 2., 2.,
       2., 2., 2., 5., 5., 5., 5., 2., 2., 2., 2., 1., 2., 2., 5., 5., 5.,
       1., 5., 2., 5., 2., 1., 2., 2., 2., 5., 2., 2., 2., 2., 2., 2., 2.,
       5., 2., 2., 1., 1., 5., 5., 2., 5., 5., 2., 2., 2., 2., 2., 2., 5.,
       2., 2., 5., 2., 2., 2., 2., 2., 5., 2., 5., 5., 2., 1., 2., 2., 5.,
       2., 1., 2., 5., 2.

In [130]:
labels_test

array([2., 3., 5., 2., 1., 2., 5., 5., 5., 2., 2., 2., 2., 2., 5., 3., 5.,
       5., 2., 5., 1., 5., 5., 2., 2., 2., 5., 2., 5., 2., 2., 5., 2., 3.,
       2., 2., 5., 5., 2., 2., 2., 5., 2., 2., 5., 5., 1., 2., 2., 5., 5.,
       2., 2., 2., 2., 5., 3., 2., 5., 5., 5., 2., 0., 5., 1., 2., 2., 5.,
       1., 5., 2., 2., 1., 2., 5., 5., 5., 5., 1., 5., 2., 2., 2., 2., 2.,
       2., 5., 2., 2., 2., 2., 2., 5., 2., 5., 2., 0., 2., 5., 2., 5., 2.,
       0., 2., 1., 2., 2., 2., 0., 5., 2., 2., 5., 2., 5., 5., 2., 2., 2.,
       5., 0., 5., 2., 2., 2., 2., 2., 2., 1., 2., 2., 2., 2., 5., 3., 2.,
       1., 2., 2., 2., 2., 2., 2., 2., 5., 1., 5., 3., 2., 2., 5., 5., 1.,
       2., 2., 2., 5., 1., 5., 3., 1., 2., 2., 2., 0., 2., 2., 5., 5., 5.,
       1., 4., 2., 5., 2., 1., 2., 0., 2., 5., 2., 5., 5., 5., 0., 2., 2.,
       2., 5., 2., 2., 5., 5., 5., 2., 5., 5., 2., 2., 2., 2., 2., 2., 2.,
       2., 5., 2., 2., 5., 2., 5., 5., 5., 2., 5., 5., 5., 3., 1., 2., 1.,
       2., 1., 2., 5., 5.

In [131]:
confusion=metrics.confusion_matrix(labels_test,y_pred)
confusion

array([[  0,   1,  14,   0,   0,   3],
       [  0,  16,  11,   0,   0,  17],
       [  0,   4, 177,   0,   0,  48],
       [  0,   2,   3,   0,   0,  10],
       [  0,   0,   3,   0,   0,   2],
       [  0,   2,  63,   0,   0, 100]])

In [133]:
np.sum(np.diag(confusion) / np.sum(confusion,axis=1))/6

0.2904371223148516

SVM TEST

In [134]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['linear','rbf', 'poly', 'sigmoid']}
grid=GridSearchCV(svm.SVC(),param_grid,refit=True,verbose=2)
grid.fit(features_train,labels_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   0.1s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   0.1s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   0.1s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   0.1s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   0.1s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.4s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.4s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   0.7s
[CV] END ........................C=0.1, gamma=1

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             verbose=2)

In [135]:
grid_predictions = grid.predict(features_test)
confusion_m=metrics.confusion_matrix(labels_test,grid_predictions)
confusion_m

array([[  0,   1,  11,   0,   0,   6],
       [  0,  17,  11,   1,   0,  15],
       [  2,   7, 165,   1,   1,  53],
       [  0,   2,   2,   0,   0,  11],
       [  0,   0,   3,   0,   0,   2],
       [  3,  11,  55,   1,   0,  95]])

In [137]:
np.sum(np.diag(confusion_m) / np.sum(confusion_m,axis=1))/6

0.28044087159807685